In [ ]:
from bs4 import BeautifulSoup
import requests
import random
import time

USER_AGENT_LIST = [
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1",
    "Mozilla/5.0 (X11; CrOS i686 2268.111.0) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.57 Safari/536.11",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1092.0 Safari/536.6",
    "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1090.0 Safari/536.6",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; 360SE)",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
    "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
    "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.0 Safari/536.3",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24",
    "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24"
]

START_ITEM = 0
PARSE_ITEM = 600    # To be changed. Maximum is 12150.
PARSED_GROUP = []

In [ ]:
def parse_page(url):
    headers = {'user-agent':random.choice(USER_AGENT_LIST)}
    soup = BeautifulSoup(requests.get(url,headers=headers).text,features='lxml')

    posts = soup.find_all(name="div",attrs={"class":"channel-item"})
    assert len(posts) > 0
    
    for post in posts:
        like = post.find(name='div',attrs={"class":"likes"}).text[:-2]
        title = post.find(name='h3').text
        group = post.find(name='span',attrs={"class":"from"}).a
        group_name, group_id = group.text, group['href'].split('/')[-2]
        if group_id in PARSED_GROUP:
            continue
        
        PARSED_GROUP.append(group_id)
        writer.writerow([title, group_name, group_id, like, -1])
        
        group_url = "https://www.douban.com/group/{}/?type=essence#topics".format(group_id)
        get_essence_posts(group_url, group_name, group_id)
        time.sleep(2)

In [ ]:
def get_essence_posts(group_url, group, group_id):
    headers = {'user-agent':random.choice(USER_AGENT_LIST)}
    soup = BeautifulSoup(requests.get(group_url,headers=headers).text,features='lxml')
    
    posts = soup.find_all(name='tr',attrs={"class":""})
    for post in posts:
        Title = post.find(name='td',attrs={"class":"title"}).a
        title, post_url = Title.text.strip(), Title['href']
        reply = post.find(name='td',attrs={"class":"r-count"}).text
        
        writer.writerow([title, group, group_id, -1, reply])

In [ ]:
import csv

with open("./posts_0.csv", 'w', encoding='utf-8', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['title','group','group_id','like','reply'])

    for i in range(START_ITEM,PARSE_ITEM,30):
        print("\n*****************Start page {}/{}*****************\n".format(i//30, PARSE_ITEM//30))
        url = "https://www.douban.com/group/explore?start={}".format(i)
        try:
            parse_page(url)
            time.sleep(2)
        except:
            print("\n\n\n*****************Waiting for 600s*****************\n\n\n")
            time.sleep(600)
            pass

## `For data exploration`

In [ ]:
import pandas as pd

file = pd.read_csv('posts.csv')
groups = file['group'].drop_duplicates()

In [ ]:
data = file[file['like'] != -1].copy()
data.sort_values(by='like', axis=0, ascending=False, inplace=True)
data[:50]

In [ ]:
data = file[file['reply'] != -1].copy()
data.sort_values(by='reply', axis=0, ascending=False, inplace=True)
data[:50]

## `Get more data from target groups`

In [ ]:
def parse_group(base_url, ID):
    for i in range(0, 800, 25):
        try:
            url = base_url + "?start={}&type=essence".format(i)

            headers = {'user-agent':random.choice(USER_AGENT_LIST)}
            soup = BeautifulSoup(requests.get(url,headers=headers).text,features='lxml')
            time.sleep(2)

            posts = soup.find_all(name="tr",attrs={"class":""})
            assert len(posts) > 0

            for post in posts:
                title = post.find(name='td',attrs={"class":"title"}).a
                href, title_text = title['href'], title.text.strip()
                num_reply = post.find(name='td',attrs={'class':'r-count'}).text.strip()
                writer.writerow([title_text, ID, href, num_reply])
        except:
            return
        
        print("Finished {}/800".format(i))